## Assignment 3 - Named Entity Recognition

In this assignment, we are going to build a Named Entity Recognition model. With this model, we will also tag new data.

More on Named Entity Recognition:

https://blog.paralleldots.com/data-science/named-entity-recognition-milestone-models-papers-and-technologies/

https://blog.paralleldots.com/product/applications-named-entity-recognition-api/

### Steps:

**1. Import the data**

**2. Build the model**

**3. Pick a dataset to run the model on**

**4. Build a function to load new data and print the tags**

Your web application will load small sections of text (such as tweets or headlines) and from that, you will tag the text based on the presence of named entities.

*What you will be graded on:*

1. Ability to build a model on word and tag data

2. Ability to use the model to predict on new data and display that prediction

*The model will be based on:*
1. Embeddings from words
2. Embeddings from tag inputs

### Step 1: Importing the data

Below is some code to get you started. As in the part of speech tagging example, you will have to write code to:

0. Split your data into a train/test set (Do a 80/20 or 90/10 split since we'll be later applying this model to an entirely separate set of data)
1. Find the set of all words
2. Find the set of all tags
3. Make a dictionary of words to index and entity tag to index

In [1]:
import pandas as pd
import numpy as np

### NER DATASET IS FOUND IN THE COURSE REPO
data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")
data.tail(10)

,Sentence #,Word,POS,Tag
1048565,Sentence: 47958,impact,NN,O
1048566,Sentence: 47958,.,.,O
1048567,Sentence: 47959,Indian,JJ,B-gpe
1048568,Sentence: 47959,forces,NNS,O
1048569,Sentence: 47959,said,VBD,O
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


### Step 1a: Formatting the data
Data will need to be

1. Indexed
2. Limited by vocabulary (ie replace tokens with UNKNOWN if they are too rare, come up with a reasonable limit based on your survey of the data and also model performance)
3. Padded

In [2]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
        
getter = SentenceGetter(data)
sentences = getter.sentences

In [11]:
X = [[w[0] for w in s] for s in sentences]
y = [[w[2] for w in s] for s in sentences]

In [12]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [5]:
def make_lexicon(token_seqs, min_freq=3, start = True):
    # First, count how often each word appears in the text.
    token_counts = {}
    for seq in token_seqs:
        for token in seq:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1

    # Then, assign each word to a numerical index. Filter words that occur less than min_freq times.
    lexicon = [token for token, count in token_counts.items() if count >= min_freq]
    # Indices start at 1. 0 is reserved for padding, and 1 is reserved for unknown words.
    if start:
        lexicon = {token:idx + 2 for idx,token in enumerate(lexicon)}
    else:
        lexicon = {token:idx for idx,token in enumerate(lexicon)}
    if min_freq>0:
        lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(len(lexicon)))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

def tokens_to_idxs(token_seqs, lexicon):
    idx_seqs = [[lexicon[token] if token in lexicon else lexicon['<UNK>'] for token in token_seq]  
                                                                     for token_seq in token_seqs]
    return idx_seqs

In [6]:
words_lexicon = make_lexicon(X_tr, min_freq=3)
tags_lexicon = make_lexicon(y_tr, min_freq=0, start=False)

LEXICON SAMPLE (14536 total items):
{u'Negroponte': 7130, u'Andreas': 10795, u'Foundation': 27, u'granting': 10799, u'both': 12257, u'1,800': 2, u'eligible': 10800, u'yellow': 3484, u'Group-B': 11913, u'revelers': 12, u'Pronk': 3, u'jihad': 10798, u'Olympics': 7132, u'Safarova': 4, u'hanging': 5, u'Sugar': 3485, u'centimeter': 22, u'al-Mashhadani': 6, u'aggression': 7135, u'marching': 3487}
LEXICON SAMPLE (17 total items):
{u'I-art': 1, u'B-gpe': 2, u'B-art': 3, u'I-per': 6, u'I-tim': 4, u'B-tim': 5, u'O': 0, u'B-geo': 7, u'I-org': 8, u'I-geo': 9, u'B-nat': 16, u'I-eve': 11, u'B-eve': 12, u'I-gpe': 13, u'B-org': 14, u'I-nat': 15, u'B-per': 10}


In [13]:
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
max_len = 75
n_tags = len(tags_lexicon)
n_words = len(words_lexicon)

X_tr = tokens_to_idxs(X_tr, words_lexicon)
X_tr = pad_sequences(maxlen=max_len, sequences=X_tr, padding="post", value=0)

y_tr = tokens_to_idxs(y_tr, tags_lexicon)
y_tr = pad_sequences(maxlen=max_len, sequences=y_tr, padding="post", value=tags_lexicon["O"])
y_tr = [to_categorical(i, num_classes=n_tags) for i in y_tr]

In [8]:
X_tr.shape

(43163, 75)

### Step 2. Build the model

Here we will build a Bidirectional LSTM-CRF model using the `Bidirectional` function from Keras and `CRF` function from Keras-contrib

**Documentation and source code:**

https://keras.io/layers/wrappers/#bidirectional

https://github.com/keras-team/keras-contrib

Fit your model with a validation split of 0.1, feel free to use as many epochs as you like. Base your predictions both from the input words **and** the tags from previous words like in the POS example.

After building your model, grade your performance on your test set, both by comparing your predicted output to the actual (*at least 3 examples*) and calculate the averaged precision and recall for your tags.

In [16]:
print(X_tr[0])

[13743  6798 12013  5324  6550  1541  6576  3937 12013 11094  6650  2598
  4621 11814  2236  7573  4148  9477  3045  6550  6647 12177   676 13249
   150     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0]


In [17]:
max_len

75

In [14]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

input = Input(shape=(max_len,))

model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding

model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM

model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer

crf = CRF(n_tags)  # CRF layer

out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 75, 20)            290740    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 75, 100)           28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 75, 17)            1190      
Total params: 325,380
Trainable params: 325,380
Non-trainable params: 0
_________________________________________________________________


In [21]:
y_tr[0]

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [15]:
history = model.fit(X_tr, np.array(y_tr), batch_size=256, epochs=5,
                    validation_split=0.1, verbose=1)

Train on 38846 samples, validate on 4317 samples
Epoch 1/5
 1280/38846 [..............................] - ETA: 3:00 - loss: 11.2231 - viterbi_acc: 0.0282

KeyboardInterrupt: 

### Step 3. Pick a dataset

Pick a dataset that has short text, similar to the sentences you just tagged. Headlines and tweets are good choices.

https://www.kaggle.com/datasets?sortBy=relevance&group=public&search=news&page=1&pageSize=20&size=all&filetype=all&license=all

In [46]:
test_sentence = ["Hawking", "was", "a", "Fellow", "of", "the", "Royal", "Society", ",", "a", "lifetime", "member",
                 "of", "the", "Pontifical", "Academy", "of", "Sciences", ",", "and", "a", "recipient", "of",
                 "the", "Presidential", "Medal", "of", "Freedom", ",", "the", "highest", "civilian", "award",
                 "in", "the", "United", "States", "."]

In [51]:
tags_ind = {v:k for (k,v) in tags_lexicon.items()}

In [52]:
x_test_sent = pad_sequences(sequences=[[words_lexicon.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags_ind[pred]))

Word           ||Prediction
Hawking        : I-art
was            : O    
a              : O    
Fellow         : I-art
of             : O    
the            : O    
Royal          : B-org
Society        : I-org
,              : O    
a              : O    
lifetime       : O    
member         : O    
of             : O    
the            : O    
Pontifical     : I-art
Academy        : B-org
of             : I-org
Sciences       : I-org
,              : O    
and            : O    
a              : O    
recipient      : O    
of             : O    
the            : O    
Presidential   : O    
Medal          : I-art
of             : O    
Freedom        : B-org
,              : O    
the            : O    
highest        : O    
civilian       : O    
award          : O    
in             : O    
the            : O    
United         : B-geo
States         : I-geo
.              : O    


### Step 4. Tag your new data!

Create a modification to the **ent_tagger function** that combines words and tags from your original dataset. Now allow the function to also load new text from your new data set, and output the tags predicted from your trained model alongside the text. Make your function load five random texts from your data and output the tagged text.